[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset

Let's first of all have a look at the data. You can download the file `labeledTrainData.tsv` on the [Kaggle website of the competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/data), or on our [Google Drive](https://drive.google.com/file/d/1a1Lyn7ihikk3klAX26fgO3YsGdWHWoK5/view?usp=sharing)


In [10]:
# Import pandas, numpy
import numpy as np
import pandas as pd

In [11]:
# Read dataset with extra params sep='\t', encoding="latin-1"
df = pd.read_csv('labeledTrainData.tsv',sep='\t', encoding="latin-1")

In [12]:
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


## 2. Preprocessing

In [24]:
from collections import Counter

vocab = Counter()
for twit in df.review:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(20)

import nltk
nltk.download('stopwords')
# stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

vocab_reduced = Counter()
# Go through all of the items of vocab using vocab.items() and pick only words that are not in 'stop_words' 
# and save them in vocab_reduced
for w, c in vocab.items():
    if not w in stop_words:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chung\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[('movie', 44030),
 ('film', 40147),
 ('one', 26788),
 ('like', 20274),
 ('good', 15140),
 ('would', 13490),
 ('time', 12723),
 ('even', 12649),
 ('story', 11975),
 ('really', 11733),
 ('see', 11469),
 ('well', 10662),
 ('much', 9765),
 ('could', 9414),
 ('get', 9310),
 ('bad', 9301),
 ('people', 9285),
 ('also', 9156),
 ('first', 9061),
 ('great', 9058)]

In [25]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    text = text.lower()
    text = re.sub('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});','',text)  # for removal of html tags
    text = re.sub(r"can't","cannot",text)
    text = re.sub(r"shan't","shall not",text)
    text = re.sub(r"won't","will not",text)
    text = re.sub(r"n't"," not",text) # see the space before not. 
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"let's","let us",text)
    text = re.sub(r"'re"," are",text)
    text = re.sub(r"'s"," ",text)  # space because we dont know the tense , it can be is/has anything.
    text = re.sub(r"'ve"," have",text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"[^a-zA-Z]"," ",text)
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text) # Removing digits
    text = re.sub('\W', ' ', text)  # If the comment/word does not contain any alphabets
    text = re.sub('\s+', ' ', text) # If there are more than one whitespace simultenously, then replace them by only 1 whitespace and also replace the punctuation marks
    text = text.strip(' ') # Removing leading and trailing white spaces
    return text

In [26]:
df['review'] = df['review'].apply(lambda text: preprocessor(text))

In [27]:
df['review'].head()

0    with all this stuff going down at the moment w...
1    the classic war of the worlds by timothy hines...
2    the film starts with a manager nicholas bell g...
3    it must be assumed that those who praised this...
4    superbly trashy and wondrously unpretentious e...
Name: review, dtype: object

In [28]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer

porter = PorterStemmer()

# Split a text into list of words
def tokenizer(text):
    return text.split()

# Split a text into list of words and apply stemming technic
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]


In [29]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split
X = df['review']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [ ]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words=stop_words,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

C:\Users\Chung\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'could', 'doe', 'dure', 'ha', 'hi', 'might', 'must', 'need', 'onc', 'onli', 'ourselv', 'shall', 'themselv', 'thi', 'veri', 'wa', 'whi', 'would', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


## 4. Evaluate Model

In [32]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model
# Your code here
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.887
confusion matrix:
 [[2167  336]
 [ 229 2268]]
classification report:
               precision    recall  f1-score   support

           0       0.90      0.87      0.88      2503
           1       0.87      0.91      0.89      2497

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000



## 5. Export Model 

In [ ]:
# Using pickle to export our trained model
import pickle
import os

import pickle
import os

pickle.dump(clf, open('logisticRegression.pkl', 'wb'))